In [1]:
import extracting_data
import extracting_metadata
import neural_network
import predict_nodule
import display_results

Using TensorFlow backend.


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
%notebook "C:\\Users\\Animesh Garg\\Lung Cancer Detection\\SPIE-AAPM Lung CT Challenge\\code\\Pipeline.ipynb" 

In [4]:
import setting
import os

# Extracting Metadata

The training Set consists of a list of dicom files corrosponding to each patient which has the image of each 2D slice of the lung and has a set of additional attributes such as Patient Age, Sex, The PixelSpacing and a lot more . So in extracting metadata I pull up the Age and Sex of the patient and make a new csv file along with the nodule position and the malignancy of the nodule

In [ ]:
extracting_metadata.extract_metadata(setting.TRAINING_SET)
extracting_metadata.extract_metadata(setting.TEST_SET)

# Extracting the lung images

This is the step where a lot of EDA and data preprocessing was done. All the slices of the lung of a particular patient are in one folder. First we extract the pixel values of each slice of the lung of a patient and sort it according to the depth of the slice which is given by the attribute "InstanceNumber". After that we transform the pixel values from pixels in their stored on disk representation to their in memory representation. This is done by a linear transformation which has two attributes , RescaleSlope and RescaleIntercept which are the attributes specified for each dicom file.

                                U = m*SV + b

Here, m is the slope, b is the intercept, SV is the stored on disk pixel values and U is the in memory pixel values.

Why have a different representation on disk versus in memory? A different range of values may be possible with the representation stored on disk versus in memory. An example: CT images, whose pixel values are measured in Hounsfield units, which can have negative values, are commonly stored with an unsigned integer. As a consequence, it is common for CT DICOM files to have a negative intercept.  The linear scaling is also applied in cases where a pixel may have a large range of values while storing the values with as few bits as possible and while avoiding quantization errors. 

After getting the exact pixel values of the image , we resample the image so that the voxel cube has the dimension of 1 * 1 * 1 mm. Each dicom file has an attribute PixelSpacing which defines the physical distance between the pixel. It is defined as a tuplpe in which the first value is the difference between the rows and the second value is the difference between the columns. 
Also, SliceThickness defines the physical width of each 2D slice extracted. Many different voxel cube volumns were tried but 1 * 1 * 1 mm cube gave the best result

So, we extract each patient image as a 3D array with dimensions (Z * Y * X) and we extract two 32 * 32 *32 CUBE out of each image, one having the nodule and one that doesn't have the nodule. Each 2D slice is a 512 * 512 image and detecting only one point out of the whole 3D array and predicting it's malignancy is like picking a needle out of a haystack . So to simplify the problem we extract small cubes so that the problem becomes easies and more targeted


In [ ]:
extracting_data.generate_data(setting.TRAINING_SET, "Train")

# Data Augmentation

The SPIE-AAPM Lung Dataset has only patient images for 70 patients, 60 in Training Set and 10 in Test Set. So the lack of data becomes a huge problem because the problem is hard and complicated and the dataset is just not big enough to learn anything meaningful. Also there is a big chance of overfitting with small datasets. So to tackle the challange for each image, I used different types of augmentation techniques such as Scaling, Translation, Rotation and Flipping. So now for each patient, I had 7 different images. So now my training Dataset increased from 60 to 420 which still isn't a lot, but it's still better then 60. 

# The neural network

I used a varient of the C3D architecture which uses 3D Convolutions and Pooling Layers to take into account the Spatiotemporal features of the 3D image. I used a set of 3D Convolution layers with pooling so that the output shape remains the same. I downsampled the cube by using the 3D Pooling layers. The output of the network are two values, one predicting the probability of a nodule being present in the 3D image and the other for the malignancy of the nodule in that image. I first tried to use YOLO loss function due to it's resemblence with the YOLO algorithm but that proved to be overkill for such a small dataset. I tried tuning the various hyperparamters, used different optimizers, but the problem was the small dataset. So eventually to get good results I used pretrained weights that I got from the 2nd prize winner of the Data Science Bowl Competition 2017. 

In [3]:
model = neural_network.neural_network()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 32, 1 0                                            
__________________________________________________________________________________________________
average_pooling3d_1 (AveragePoo (None, 16, 32, 32, 1 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv3D)                  (None, 16, 32, 32, 6 1792        average_pooling3d_1[0][0]        
__________________________________________________________________________________________________
pool1 (MaxPooling3D)            (None, 16, 16, 16, 6 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Con

# Predicting the nodules and their malignancy in Test Set

For each patient in the test set, I took a 32 * 32 * 32 CUBE chunk convering the whole test image. I passed each CUBE chuck through the neural network and saved the co-ordinates of the center of the CUBE and the probability of malignancy where the network predicted the probability of a nodule being present above a certain threshold (0.3). I save the data for each patient in a csv file.  

In [ ]:
model_path = os.path.join(setting.MODEL_WEIGHTS, "model_luna16_full__fs_best.hd5")
predict_nodule.predict_nodules(setting.TEST_SET, model_path, csv_target_path)

# Displaying the predictions

Now, for each patient, we have it's corrosponding predicted nodule position and it's probability of malignancy. For each 2D slice, I make a 24-bit colored image and color the 10 * 10 cube around the predicted nodule position. And I make the set of 24-bit colored image for each 2D slice into a GIF having 10 FPS. The results are stored in the results folder.

In [ ]:
display_results.make_gifs(setting.TEST_SET, setting.CSV_TARGET, color_intensity=1000, color='red')

![SegmentLocal](C:\Users\Animesh Garg\Lung Cancer Detection\SPIE-AAPM Lung CT Challenge\result\ct-training-be "segment")